In [9]:
from nano_graphrag import GraphRAG, QueryParam 
import nest_asyncio
nest_asyncio.apply()
from sentence_transformers import SentenceTransformer
import numpy as np
from nano_graphrag import GraphRAG
from dotenv import load_dotenv
load_dotenv()  # .env 파일에 OPENAI_API_KEY가 포함되어 있어야 함
from nano_graphrag._llm import openai_complete_if_cache
from sentence_transformers import SentenceTransformer
from nano_graphrag import GraphRAG, QueryParam  # QueryParam 별도 import


async def my_custom_llm(prompt, system_prompt=None, history_messages=[], **kwargs):
    return await openai_complete_if_cache(
        "gpt-4o-mini",  # 원하는 모델
        prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        **kwargs
    )


# 안정적으로 로딩 가능한 한국어 SBERT 모델
model_name = 'jhgan/ko-sroberta-multitask'
model = SentenceTransformer(model_name)

# nano-graphrag에서 요구하는 래핑
def wrap_embedding_func_with_attrs(embedding_dim, max_token_size):
    def decorator(func):
        func.embedding_dim = embedding_dim
        func.max_token_size = max_token_size
        return func
    return decorator

@wrap_embedding_func_with_attrs(embedding_dim=768, max_token_size=512)
async def local_embedding_func(texts: list[str]) -> np.ndarray:
    """
    한국어 문장 리스트를 입력받아 SBERT 임베딩을 반환합니다.
    """
    return model.encode(texts, convert_to_numpy=True)
graph_func = GraphRAG(
    working_dir="./graphrag",
    best_model_func=my_custom_llm,
    embedding_func=local_embedding_func,

    # ✅ 청크 사이즈를 아주 크게 설정해서 "절대 자르지 않게" 함
    chunk_token_size=100000,
    chunk_overlap_token_size=0,
)

# 1) 질의 내용 정의
query_text = (
    "나는 둥근형 얼굴이고, 직모에 굵은 모발을 가졌으며, "
    "관리 난이도 쉬운 남성 스타일을 추천해줘."
)

param_local = QueryParam(mode="local")
response_local = graph_func.query(query_text, param_local)
print("=== Local RAG Recommendation ===")
print(response_local)  # JSON 형식의 community_report_hair 결과

param_global = QueryParam(mode="global")
response_global = graph_func.query(query_text, param_global)
print("=== Global RAG Recommendation ===")
print(response_global)


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: jhgan/ko-sroberta-multitask
INFO:nano-graphrag:Load KV full_docs with 10 data
INFO:nano-graphrag:Load KV text_chunks with 10 data
INFO:nano-graphrag:Load KV llm_response_cache with 43 data
INFO:nano-graphrag:Load KV community_reports with 22 data
INFO:nano-graphrag:Loaded graph from ./graphrag/graph_chunk_entity_relation.graphml with 732 nodes, 161 edges
INFO:nano-vectordb:Load (724, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': './graphrag/vdb_entities.json'} 724 data
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.61it/s]
INFO:nano-graphrag:Using 20 entites, 5 communities, 38 relations, 9 text units
INFO:nano-graphrag:Revtrieved 22 communities
INFO:nano-graphrag:Grouping to 1 groups for global search


=== Local RAG Recommendation ===
## 둥근 얼굴형과 직모의 특성

둥근 얼굴형은 부드러운 윤곽선과 풍성한 볼로 특징지어지며, 이러한 얼굴형에 적합한 스타일은 얼굴의 굴곡을 자연스럽게 보완할 수 있어야 합니다. 직모와 굵은 모발을 가진 경우, 보다 정돈된 느낌의 스타일이 적합할 것입니다. 이러한 조합을 고려할 때, 관리 난이도가 쉬운 스타일을 선택하는 것이 좋습니다.

## 추천 스타일: 가르마펌

가르마펌은 둥근 얼굴형에 잘 어울리는 남성 스타일 중 하나입니다. 이 스타일은 깔끔하고 세련된 인상을 주며, 특히 직모와 굵은 모발에 적합합니다. **가르마펌**의 특징은 각진 얼굴형을 부드럽게 커버할 수 있기 때문에, 둥근 얼굴형에도 매우 매력적입니다. 이 스타일은 볼륨이 부족한 경우에도 효과적으로 적용할 수 있으며, 고객의 얼굴형에 맞게 개인 맞춤형 시술이 가능합니다. 특히, 면접이나 소개팅 등에서 긍정적인 인상을 줄 수 있는 스타일로 추천됩니다.

## 추가 스타일링: 가일컷

또 다른 추천 스타일은 **가일컷**입니다. 이 스타일은 직모에 최적화되어 있으며, 자연스럽고 단정한 느낌을 줍니다. 관리가 쉬운 스타일이며, 이마를 드러내어 성숙하고 깔끔한 분위기를 연출할 수 있습니다. 가일컷은 다양한 얼굴형에 어울리지만, 특히 둥근 얼굴형에게 더 잘 맞기 때문에 좋습니다. 가일컷은 앞머리의 길이를 조정할 수 있어, 개인의 취향에 맞게 스타일링할 수 있는 장점이 있습니다.

## 관리의 용이함

가르마펌과 가일컷 모두 관리가 용이하며, 바쁜 현대인들에게 특히 적합한 선택입니다. 스타일링에 필요한 제품, 예를 들어 가벼운 왁스나 헤어 스프레이를 이용하면 쉽게 원하는 스타일을 유지할 수 있습니다. 이러한 스타일들은 시간의 제약이 있는 남성들에게 매우 바람직한 옵션이 될 것입니다.

위의 스타일 조합을 통해 둥근 얼굴형과 직모, 굵은 모발에서 매력적인 이미지를 연출할 수 있을 것입니다. 원하는 스타일을 선택하고 이와 적합한 관리법을 통해 자신만의 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:nano-graphrag:JSON data successfully extracted.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Global RAG Recommendation ===
## 추천 스타일

당신의 둥근 얼굴형과 직모, 굵은 모발에 적합한 스타일을 고려할 때, 두 가지 스타일이 특히 추천됩니다: **가르마펌**과 **가일컷**입니다.

### 가르마펌

가르마펌은 각진 얼굴형과 둥근 얼굴형 모두에 적합하여, 부드러운 인상을 줄 수 있습니다. 이 스타일은 직모를 가진 남성이 자연스러운 볼륨을 추가할 수 있도록 도와줍니다. 때문에 둥근 얼굴형을 더욱 매력적으로 보이게 할 수 있는 효과가 있습니다. 특히, 관리가 용이한 장점이 있어 바쁜 일상 속에서도 편리하게 유지할 수 있습니다.

### 가일컷

가일컷은 굵은 모발에 잘 어울리며, 깔끔한 느낌을 주는 스타일입니다. 이 스타일은 얼굴형을 보완하고, 시원한 인상을 제공하며 유지 관리가 쉽기 때문에 당신에게 적합할 것입니다. 굵은 모발을 가진 경우, 이러한 스타일링은 더욱 뚜렷한 인상을 남기는데 기여할 수 있습니다.

## 결론

결론적으로, 둥근 얼굴형과 굵은 직모를 가진 당신에게는 가르마펌과 가일컷이 좋은 선택이 될 것입니다. 이 두 스타일 모두 관리가 용이하며, 각각 특유의 매력을 가져올 수 있는 옵션임을 고려하면 좋겠습니다. 스타일을 선택하기에 앞서, 각 스타일의 시각적 효과와 편의성을 잘 비교해 보시길 권장합니다.


In [7]:
import pandas as pd 
df= pd.read_csv('hair_data/for_graph.csv')
df
# 요약 부분에 마크다운 형식 즉 *,# 등이 있으면 삭제하는 코드
import re
def remove_markdown(text):
    # 마크다운 형식의 문자를 정규 표현식으로 제거
    text = re.sub(r'[*#]', '', text)
    return text
df['요약'] = df['요약'].apply(remove_markdown)


In [8]:
docs = df["요약"].dropna().astype(str).tolist()  # 🔥 행 하나 = 하나의 청크로 사용됨
graph_func.insert(docs[25:35])
# docs

INFO:nano-graphrag:[New Docs] inserting 10 docs
INFO:nano-graphrag:[New Chunks] inserting 10 chunks
INFO:nano-graphrag:[Entity Extraction]...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:nano-graphrag:JSON data successfully extracted.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:nano-graphrag:JSON data successfully extracted.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:nano-graphrag:JSON data successfully extracted.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:nano-graphrag:JSON data successfully extracted.


INFO:nano-graphrag:Writing graph with 732 nodes, 161 edges


In [8]:
from nano_graphrag import GraphRAG, QueryParam  # QueryParam 별도 import

# 1) 질의 내용 정의
query_text = (
    "나는 둥근형 얼굴이고, 직모에 굵은 모발을 가졌으며, "
    "관리 난이도 쉬운 남성 스타일을 추천해줘."
)

# 2) Local RAG 모드
param_local = QueryParam(mode="local")
response_local = graph_func.query(query_text, param_local)
print("=== Local RAG Recommendation ===")
print(response_local)  # JSON 형식의 community_report_hair 결과

# 3) Global RAG 모드
param_global = QueryParam(mode="global")
response_global = graph_func.query(query_text, param_global)
print("=== Global RAG Recommendation ===")
print(response_global)

# 4) (Optional) Naive 모드
# param_naive = QueryParam(mode="naive")
# response_naive = graph_func.query(query_text, param_naive)
# print("=== Naive RAG Recommendation ===")
# print(response_naive)


Batches: 100%|██████████| 1/1 [00:00<00:00, 22.03it/s]
INFO:nano-graphrag:Using 20 entites, 1 communities, 18 relations, 7 text units


KeyboardInterrupt: 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
